In [1]:
### IMPORTING LIBRARIES
import bs4
import pandas as pd
import requests
import re
import string

In [2]:
### BS4 oject
res = requests.get("http://www.dailytenminutes.com/p/short-stories.html")
res.raise_for_status() # checking if request was successful
readsSoup = bs4.BeautifulSoup(res.text,'html.parser')


links = []
### Getting all links
for a in readsSoup.find_all('a',href=True):
    if a.text:
        links.append(a['href'])
l = len(links)
#print(l)

### Using regex to find all links for stories
def cleaningLinks(L):
    rlinks = []
    for i in links:
        linkreg = re.search('[a-zA-Z.]+/\d+/\d+',i)
        if linkreg!=None:
            rlinks.append(i)
    return rlinks

a = cleaningLinks(links)

### Storing all links to CSV
def csvLinks(links):
    df = pd.DataFrame(links,columns=["h"])
    df["Done"] = 0
    df.to_csv("links.csv",index = False)
csvLinks(a)





In [3]:
### Extract link from csv file and mark it as done
def getLink():
    df = pd.read_csv("links.csv")
    for index,row in df.iterrows():
        if row['Done'] == 1:
            continue
        else:
            print(index,row['h'],row['Done'])
            df.loc[index,"Done"] = 1
            df.to_csv("links.csv", index=False)
            return row['h']
            break
a = getLink()


0 http://www.dailytenminutes.com/2017/04/in-class-i-teach-for-adults-i-recently.html 0


In [5]:
### Mine the story from the extracted link
bookres = requests.get(a)
res.raise_for_status()
linksSoup = bs4.BeautifulSoup(bookres.text,'html.parser')
L = []
def getStory(): 
    p = linksSoup.findAll('article')
    for r in p:
        L.append(r.get_text())
getStory()

### Cleaning/trimming the story
def cleanStory(L):
    f = re.sub(r'\\n+|\\'," ",L[0]) #removing all \n
    f1 = " ".join(f.split())
    flag = True
    while flag == True:    # removing the intro part
        for i in range(len(f1)//2):
            if f1[i] == "." and f1[i-1] == "." and f1[i-2] == ".":
                p = f1[i+1:len(f1)]
                flag = False
    return p
a = cleanStory(L)
print(a)

In a class I teach for adults, I recently did the "unpardonable." I gave the class homework! The assignment was to "go to someone you love within the next week and tell them you love them. It has to be someone you have never said those words to before or at least haven't shared those words with for a long time." Now that doesn't sound like a very tough assignment, until you stop to realized that most of the men in that group were over 35 and were raised in the generation of men that were taught that expressing emotions is not "macho." Showing feelings or crying (heaven forbid!) was just not done. So this was a very threatening assignment for some. At the beginning of our next class, I asked if someone wanted to share what happened when they told someone they loved them. I fully expected one of the women to volunteer, as was usually the case, but on this evening one of the men raised his hand. He appeared quite moved and a bit shaken. As he unfolded out of his chair (all 6'2" of him), h